# MSBD 566 - Lecture 9
## Clustering - Work with Real Data

### The data [from Data Science and Predictive Analytics by Ivo D. Dinov]

The dataset we will be using is the Divorce and Consequences on Young Adults dataset. This is a longitudinal study focused on examining the consequences of recent parental divorce for young adults (initially ages 18–23) whose parents had divorced within 15 months of the study’s rst wave (1990–91). The sample consisted of 257 White respondents with newly divorced parents. Here we have a subset of this dataset with 47 respondents in our case-studies folder, CaseStudy01_Divorce_YoungAdults_Data.csv.

**Variables**
* `DIVYEAR`: Year in which parents were divorced. Dichotomous variable with
1989 and 1990.
* Child affective relations:
  * `Momint`: Mother intimacy. Interval level data with four possible responses (1-extremely close, 2-quite close, 3-fairly close, 4- not close at all).
  * `Dadint`: Father intimacy. Interval level data with four possible responses (1-extremely close, 2-quite close, 3-fairly close, 4-not close at all).
  * `Live with mom`: Polytomous variable with three categories (1- mother only, 2-father only, 3- both parents).
* `momclose`: measure of how close the child is to the mother (1-extremely close, 2-quite close, 3-fairly close, 4-not close at all).
* `Depression`: Interval level data regarding feelings of depression in the past 4 weeks. Possible responses are 1-often, 2-sometimes, 3-hardly ever, 4-never.
* `Gethitched`: Polytomous variable with four possible categories indicating respondent’s plan for marriage (1-Marry fairly soon, 2-marry sometime, 3-never marry, 8-don’t know).

### Prepare the data

In [ ]:
# Import modules
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import MeanShift, estimate_bandwidth

In [ ]:
# Loading data
data = pd.read_csv('CaseStudy01_Divorce_YoungAdults_Data.csv')
print(data.head())
print(f"Data shape (before cleaning): {data.shape}")

# clear missing value
data = data.dropna()
print(f"Data shape (after cleaning): {data.shape}")

We have to make sure all the values are valid as described above. 

For example:
* According to the summary, DIVYEAR is actually a dummy variable (either 89 or 90). So we can probably change them to binary (0 for 1989 and 1 for 1990)
* There is 9 for `livewithmom` which is not a valid number for this variable. We have options to remove this row, or change to something that makes more sense.

In [ ]:
# check general statistics - to see any anomalies
print(data.describe())

In [ ]:
# change DIVYEAR to binary
data['DIVYEAR'] = data['DIVYEAR'].replace({89: 0, 90: 1})
print(data['DIVYEAR'].value_counts()) # just to double check

# change livewithmom = 9 to NaN and drop
data['livewithmom'] = data['livewithmom'].replace(9, np.nan)
data = data.dropna()
print(f"Data shape (after cleaning livewithmom): {data.shape}")

### Clustering Method 1: k-means

In [ ]:
# K-means clustering
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

# Standardize the data
scaler = StandardScaler()
data_scaled = scaler.fit_transform(data)

# Determine the optimal number of clusters using the silhoutte score method

silhouette_scores = []
k_range = np.arange(2, 10)
for k in k_range:
    kmeans = KMeans(n_clusters=k, random_state=42)
    cluster_labels = kmeans.fit_predict(data_scaled)
    silhouette_avg = silhouette_score(data_scaled, cluster_labels)
    silhouette_scores.append(silhouette_avg)

# Plotting the silhouette scores
plt.figure(figsize=(6, 3))
plt.plot(k_range, silhouette_scores, marker='o')
plt.xlabel('Number of clusters (k)')
plt.ylabel('Silhouette Score')
plt.title('Silhouette Score Method for Optimal k')
plt.show()

# Choosing k based on the silhouette plot
optimalK = k_range[np.argmax(silhouette_scores)]
print(f"The optimal number of clusters (k) is: {optimalK}")

In [ ]:
# running K-means
kmeans = KMeans(n_clusters=2, random_state=42)
data['Cluster_kmeans'] = kmeans.fit_predict(data_scaled)
print(data.head())

print("Cluster counts:")
print(data['Cluster_kmeans'].value_counts())

# Visualizing clusters on plots based on two features - live with mom vs close to mom
plt.figure(figsize=(6, 3))
markerlist = ['o', 's', 'D', 'X', 'P']
# each cluster will have different shapes and colors
sns.scatterplot(x=data.columns[5], y=data.columns[1], hue='Cluster_kmeans', data=data, palette='Set1', markers=markerlist[:len(data['Cluster_kmeans'].unique())], style='Cluster_kmeans')
plt.title('K-means Clustering Results')
plt.show()

### Clustering Method 2: DBSCAN

In [ ]:
# Using DBSCAN for clustering
from sklearn.cluster import DBSCAN
# Standardize the data
scaler = StandardScaler()
data_scaled = scaler.fit_transform(data)

# calculate eps for dbscan
bandwidth = estimate_bandwidth(data_scaled, quantile=0.2, n_samples=45)
print(f"Estimated bandwidth (eps) for DBSCAN: {bandwidth}")

# running DBSCAN
dbscan = DBSCAN(eps=bandwidth, min_samples=5)  # eps and min_samples need to be tuned
data['Cluster_dbscan'] = dbscan.fit_predict(data_scaled)
print(data.head())

print("Cluster counts:")
print(data['Cluster_dbscan'].value_counts())

# Visualizing DBSCAN clusters on plots based on two features - live with mom vs close to mom
markerlist = ['o', 's', 'D', 'X', 'P']
plt.figure(figsize=(6, 3))
# each cluster will have different shapes and colors
sns.scatterplot(x=data.columns[5], y=data.columns[1], hue='Cluster_dbscan', data=data, palette='Set1', markers=markerlist[:len(data['Cluster_dbscan'].unique())], style='Cluster_dbscan')
plt.title('DBSCAN Clustering Results')
plt.show()



### Clustering Method 3: Mean-Shift

In [ ]:
# mean shift clustering

# Standardize the data
scaler = StandardScaler()
data_scaled = scaler.fit_transform(data)

# Estimate bandwidth
bandwidth = estimate_bandwidth(data_scaled, quantile=0.2)
print(f"Estimated bandwidth: {bandwidth}")

# running Mean Shift
mean_shift = MeanShift(bandwidth=bandwidth, bin_seeding=True)
data['Cluster_meanshift'] = mean_shift.fit_predict(data_scaled)
print(data.head())

print("Cluster counts:")
print(data['Cluster_meanshift'].value_counts())

# Visualizing Mean Shift clusters on plots based on two features - live with mom vs close to mom
markerlist = ['o', 's', 'D', 'X', 'P']
plt.figure(figsize=(6, 3))
# each cluster will have different shapes and colors
sns.scatterplot(x=data.columns[5], y=data.columns[1], hue='Cluster_meanshift', data=data, palette='Set1', markers=markerlist[:len(data['Cluster_meanshift'].unique())], style='Cluster_meanshift')
plt.title('Mean Shift Clustering Results')
plt.show()

### Comparing the methods

In [ ]:
# Comparing the methods
print("Cluster counts:")
print(data[['Cluster_kmeans', 'Cluster_dbscan', 'Cluster_meanshift']].nunique())

# set the dataframe to compare each method 
df = {'Kmeans': data['Cluster_kmeans'],
      'DBSCAN': data['Cluster_dbscan'],
      'MeanShift': data['Cluster_meanshift']}

# cross tab
ctab = pd.crosstab(df['DBSCAN'], df['MeanShift'])
print(ctab)

# Create heatmap to compare clustering results
plt.figure(figsize=(5, 4))
sns.heatmap(ctab, annot=True, square=True, fmt='d')
plt.title("Cluster Cross-Tabulation (Kmeans vs DBSCAN)")
plt.xlabel("DBSCAN Clusters")
plt.ylabel("MeanShift Clusters")
plt.show()